In [2]:
words = open('names.txt', 'r').read().splitlines()
len(words)

32033

In [3]:
bigram_model = {}

for w in words:
    chars = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chars, chars[1:]):
        bigram = (ch1, ch2)
        bigram_model[bigram] = bigram_model.get(bigram, 0) + 1

In [4]:
import torch

In [5]:
N = torch.zeros((27, 27), dtype=torch.int32)

chars = sorted(list(set(''.join(words))))
str_to_int = {s: i+1 for i, s in enumerate(chars)}
str_to_int['.'] = 0
int_to_str = {i: s for s, i in str_to_int.items()}

for w in words:
    chars = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chars, chars[1:]):
        i1 = str_to_int[ch1]
        i2 = str_to_int[ch2]

        N[i1, i2] += 1

In [9]:
# Efficiency, don't normalize every loop
P = N.float()
P /= P.sum(1, keepdim=True)

In [11]:
g = torch.Generator()
names = 5

for _ in range(names):
    new_name = ''
    char = '.'

    while True:
        p = P[str_to_int[char]]

        # p = N[str_to_int[char]].float()
        # p /= p.sum()    

        char = int_to_str[torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()]
        new_name += char

        if char == '.':
            break

    new_name = new_name[:-1]
    print(new_name)

kodoleiyn
tviaya
kawia
jalon
heliezi


In [23]:
'''
Log-likelihood

Kind of like a loss function for this kind of modeling
Log of the probability that the dataset is reproduced by the model
Usually negated to observe the convention of minimizing loss functions
'''
log_likelihood = 0.0
n = 0

for w in words:
    chars = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chars, chars[1:]):
        i1 = str_to_int[ch1]
        i2 = str_to_int[ch2]

        prob = P[i1, i2]
        log_likelihood += torch.log(prob)
        n += 1
        

-log_likelihood.item()/n

2.454094088872915